<a href="https://colab.research.google.com/github/nokomoro3/book-ml-transformers/blob/main/ml-transformers-chap04-multilingal-ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 多言語の固有表現認識

- 事前学習済みモデルは、英語・ドイツ語・ロシア語・中国語などの「高リソース」言語に偏って存在する傾向がある。
- またエンジニアリングチームとしても複数の言語のモデルを保守することは工数がかかる。
- そのため多言語対応したTransformerを用いることができる。
- 多言語対応したTransformerの特徴
  - 事前学習としてマスク言語モデルを学習するが100以上の言語で同時に学習される。
  - ある言語でファインチューニングされたモデルを別の言語でも適用できる、ゼロショット異言語間転移を可能にする。
  - これらのモデルは、「コードスイッチング」（１つの会話で話者が２つ以上の言語や方言を使い分けること）にも適している。
- 本章では、XLM-RoBERTaをファインチューニングすることで、複数の言語のNERを実施する方法を紹介する。
- NERの用途
  - 文書の分析、検索エンジンの品質向上、コーパスからの構造化データの構築など
- 本章の用途としては、４つの公用語を持つスイスが拠点の顧客に対してNERを実施する。

In [20]:
# Uncomment and run this cell if you're on Colab or Kaggle
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements()

Cloning into 'notebooks'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 422 (delta 72), reused 68 (delta 68), pack-reused 343
Receiving objects: 100% (422/422), 25.01 MiB | 24.79 MiB/s, done.
Resolving deltas: 100% (195/195), done.
/content/notebooks/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [21]:
#hide
from utils import *
setup_chapter()

Using transformers v4.11.3
Using datasets v1.16.1


## 4.1 データセット

- 多言語エンコーダの言語間遷移評価(XTREME: Cross-lingal TRansfer Evaluation for Multilingal Encoders)ベンチマークの、WikiANNまたはPAN-Xを使用する。
  - [XTREME: A Massively Multilingual Multi-task Benchmark for Evaluating Cross-lingual Generalization (2020-03-24)](https://arxiv.org/abs/2003.11080)
- これはスイス公用語の４言語における多言語のWikipedia記事で構成される。
- 各記事は、LOC(場所)、PER(人名)、ORG(組織名)でアノテーションされ、inside-outside-beginning(IOB2)形式である。
- 以下に例を示す。

![](https://github.com/nokomoro3/book-ml-transformers/blob/a2676dc6002993ea996bddbaf3abd6571ba3d552/img/ml-transformers-chap04-multilingal-ner_2022-08-29-08-13-29.png?raw=1)

- IOB2形式は、B-が固有表現の先頭トークンとなり、I-がその先頭に属する連続したトークン、Oが固有表現ではないトークンでタグ付けする形式。

- 以下のように関連するデータセットを調べます。

In [22]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)} configurations")
print(xtreme_subsets)

XTREME has 183 configurations
['XNLI', 'tydiqa', 'SQuAD', 'PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn',
'PAN-X.de', 'PAN-X.el', 'PAN-X.en', 'PAN-X.es', 'PAN-X.et', 'PAN-X.eu',
'PAN-X.fa', 'PAN-X.fi', 'PAN-X.fr', 'PAN-X.he', 'PAN-X.hi', 'PAN-X.hu',
'PAN-X.id', 'PAN-X.it', 'PAN-X.ja', 'PAN-X.jv', 'PAN-X.ka', 'PAN-X.kk',
'PAN-X.ko', 'PAN-X.ml', 'PAN-X.mr', 'PAN-X.ms', 'PAN-X.my', 'PAN-X.nl',
'PAN-X.pt', 'PAN-X.ru', 'PAN-X.sw', 'PAN-X.ta', 'PAN-X.te', 'PAN-X.th',
'PAN-X.tl', 'PAN-X.tr', 'PAN-X.ur', 'PAN-X.vi', 'PAN-X.yo', 'PAN-X.zh',
'MLQA.ar.ar', 'MLQA.ar.de', 'MLQA.ar.vi', 'MLQA.ar.zh', 'MLQA.ar.en',
'MLQA.ar.es', 'MLQA.ar.hi', 'MLQA.de.ar', 'MLQA.de.de', 'MLQA.de.vi',
'MLQA.de.zh', 'MLQA.de.en', 'MLQA.de.es', 'MLQA.de.hi', 'MLQA.vi.ar',
'MLQA.vi.de', 'MLQA.vi.vi', 'MLQA.vi.zh', 'MLQA.vi.en', 'MLQA.vi.es',
'MLQA.vi.hi', 'MLQA.zh.ar', 'MLQA.zh.de', 'MLQA.zh.vi', 'MLQA.zh.zh',
'MLQA.zh.en', 'MLQA.zh.es', 'MLQA.zh.hi', 'MLQA.en.ar', 'MLQA.en.de',
'MLQA.en.vi', 'MLQA.en.zh', 'MLQA.en.en', 

- 多くのデータがまだヒットするため、PAN-X関連に絞ってみます。

In [23]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
print(f"XTREME:PAN-X has {len(panx_subsets)} configurations")
print(panx_subsets)

XTREME:PAN-X has 40 configurations
['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn', 'PAN-X.de', 'PAN-X.el',
'PAN-X.en', 'PAN-X.es', 'PAN-X.et', 'PAN-X.eu', 'PAN-X.fa', 'PAN-X.fi',
'PAN-X.fr', 'PAN-X.he', 'PAN-X.hi', 'PAN-X.hu', 'PAN-X.id', 'PAN-X.it',
'PAN-X.ja', 'PAN-X.jv', 'PAN-X.ka', 'PAN-X.kk', 'PAN-X.ko', 'PAN-X.ml',
'PAN-X.mr', 'PAN-X.ms', 'PAN-X.my', 'PAN-X.nl', 'PAN-X.pt', 'PAN-X.ru',
'PAN-X.sw', 'PAN-X.ta', 'PAN-X.te', 'PAN-X.th', 'PAN-X.tl', 'PAN-X.tr',
'PAN-X.ur', 'PAN-X.vi', 'PAN-X.yo', 'PAN-X.zh']


- ISO 639-1 言語コードがサフィックスについている。（例えばドイツ語は`de`）
- それぞれのデータセットは、trainが20000件、validationとtestがそれぞれ10000件の合計40000件となっている。

In [24]:
from datasets import load_dataset

for l in ["de", "fr", "it", "en"]:
    print(load_dataset("xtreme", name=f"PAN-X.{l}"))

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
})


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
})


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
})


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
})


- これらを実際のスイス語にあったコーパスを作成するため、話者比率に合わせてサンプリングする。

In [25]:
from collections import defaultdict
from datasets import DatasetDict

langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059] # 話者の比率

# defaultdict(python標準)で設定すれば、キーが存在しない場合にDatasetDictを返すことが可能
panx_ch = defaultdict(DatasetDict)
panx_ch["de"]

DatasetDict({
    
})

In [26]:
for lang, frac in zip(langs, fracs):
    # 単言語コーパスをロード
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    
    # 各分割をシャッフルし、話者の割合に応じてダウンサンプリング
    for split in ds: # train, validation, testのループ
        panx_ch[lang][split] = (
            ds[split]
            .shuffle(seed=0)
            .select(
                range( int(frac * ds[split].num_rows) )
            )
        )

panx_ch

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

defaultdict(datasets.dataset_dict.DatasetDict, {'de': DatasetDict({
                 validation: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 6290
                 })
                 test: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 6290
                 })
                 train: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 12580
                 })
             }), 'fr': DatasetDict({
                 validation: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 2290
                 })
                 test: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 2290
                 })
                 train: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                  

- trainでその件数を確認してみる。

In [27]:
import pandas as pd

pd.DataFrame(
    {lang: [panx_ch[lang]["train"].num_rows] for lang in langs}
    , index=["Number of training examples"]
)

,de,fr,it,en
Number of training examples,12580,4580,1680,1180


- 最も多いドイツ語を出発点として、他の言語へのゼロショット転移を実行していく。
- 1つのサンプルの情報は以下のようになっている。

In [28]:
element = panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key}: {value}")

tokens: ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der',
'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags: [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


- ner_tagsは既に数値化されているため、Datasetオブジェクトのfeatures属性から情報を取得する。

In [29]:
for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

tokens: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags: Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER',
'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], names_file=None, id=None),
length=-1, id=None)
langs: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


In [30]:
tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG',
'B-LOC', 'I-LOC'], names_file=None, id=None)


- このClassLabelに、int2strメソッドがあるため、これを使えば変換することが可能。

In [31]:
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}

panx_de = panx_ch["de"].map(create_tag_names)

  0%|          | 0/6290 [00:00<?, ?ex/s]

  0%|          | 0/6290 [00:00<?, ?ex/s]

  0%|          | 0/12580 [00:00<?, ?ex/s]

- 結果を確認する。

In [32]:
de_example = panx_de["train"][0]
pd.DataFrame(
    [de_example["tokens"], de_example["ner_tags_str"]],
    ['Tokens', 'Tags']
)

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


- 念のためORG, LOC, PERのタグに偏りがないかを確認する。

In [33]:
from collections import Counter

# 再びdefaultdict
split2freqs = defaultdict(Counter)

for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1

pd.DataFrame.from_dict(split2freqs, orient="index")

,ORG,LOC,PER
validation,2683,3172,2893
test,2573,3180,3071
train,5366,6186,5810


## 4.2 多言語Transformer

- 多言語Transformerは単一言語のTransformerと大きな違いはなく、事前学習の際のコーパスが多言語になっている点が特徴。
- 一般的に、NERの言語間遷移ではCoNLL-2002やCoNLL-2003が良く使用される。
  - [CoNLL-2002 (Hugging Face)](https://huggingface.co/datasets/conll2002)
  - [CoNLL-2003 (Hugging Face)](https://huggingface.co/datasets/conll2003)
  - PAN-Xとの違いは、固有表現にその他を示すMISCがある点である。
- 多言語モデルは一般的に以下の評価戦略を用いる。
  - en : 英語でファインチューニングして、その他の言語を評価する
  - each : それぞれの言語でファインチューニングして、それぞれの言語を評価する
  - all : すべての言語でファインチューニングして、各言語をすべて評価する。
- 今回使用するモデル
  - XLM-RoBERTa(XLM-R)を使用する。
    - 初期の多言語TransformerはmBERTが挙げられ、BERTと同じ事前学習を実施したがXLM-Rに今はとって代わられたため。
  - XLM-Rの特徴
    - 事前学習のコーパスサイズが巨大（多言語のWikipedia記事、Web上のCommon Crawlを使用）
    - RoBERTaと同じ事前学習手法を使用
      - 特に次文予測を排除した点と、その他いくつかの改良。
    - 元となるXLMで使用されていた言語埋め込みを削除し
    - 生のテキストをトークン化するためにSentencePieceを使用
      

## 4.3 トークン化の詳細

- XLM-Rではトークン化にWordPieceではなく、100言語のテキストで学習したSentencePieceを使用。
- まずはこのトークナイザーを比較する。

In [34]:
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

In [35]:
text = "Jack Sparrow loves New York!"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()
print(bert_tokens)
print(xlmr_tokens)

['[CLS]', 'Jack', 'Spa', '##rrow', 'loves', 'New', 'York', '!', '[SEP]']
['<s>', '▁Jack', '▁Spar', 'row', '▁love', 's', '▁New', '▁York', '!', '</s>']


### 4.3.1 トークナイザーのパイプライン

- トークン化は文字列を整数列に変換する操作であるが、より正確には以下のパイプラインで処理される。

![]()

- 正規化
  - 生の文字列をきれいにするための処理
  - 空白除去、アクセント付き文字の除去、Unicode正規化、小文字化など。
  - Unicode正規化には、NFC, NFD, NFKC, NFKDなどのスキームがある。
    - [Unicode正規化 - Qiita](https://qiita.com/fury00812/items/b98a7f9428d1395fc230)

- 事前トークン化
  - サブワード分割前の、いわゆる単語トークンのこと。
  - 英語、ドイツ語などの多くのインド・ヨーロッパ語族の場合は空白が分割できる。
  - 一方これが自明ではなく決定論的ではない言語もあるため、それらは言語固有のライブラリを使用して、事前トークン化することも多い。

- トークナイザーモデル
  - コーパスを用いて学習した、サブワード分割モデルを適用する。
  - BPE, Unigram, WordPieceなどいくつかのサブワードトークン化アルゴリズムが存在する。

- 後処理
  - 特殊なトークン、[CLS]や[SEP]などを追加する処理などが挙げられる。
  - XLM-Rの場合、`<s>`や`</s>`が該当する。


### 4.3.2 SentencePiece トークナイザー

- Unigramと呼ばれるサブワード分割に基づき、入力テキストをUnicode文字の系列としてエンコードする。
- これによりアクセントや句読点、空白文字に依存しないため、多言語モデルに適している。
- また空白にはLower One Quarter Blockが割り当てられいる。
  - 例えば以下の`Jack`の手前にあるものがU+2581のLower One Quarter Blockである。
- これにより事前トークナイザーに依存せずに系列を元の状態に戻すことができる。
  - 通常、`!`の前には空白がないことが空白と見分けがつくため分かる。

In [36]:
xlmr_tokens

['<s>', '▁Jack', '▁Spar', 'row', '▁love', 's', '▁New', '▁York', '!', '</s>']

- ちなみに以下でコードポイントがわかる。

In [37]:
hex(bytes(xlmr_tokens[1][0], encoding='utf-16-be')[0]), hex(bytes(xlmr_tokens[1][0], encoding='utf-16-be')[1]) # BEの場合
# hex(bytes(xlmr_tokens[1][0], encoding='utf-16')[3]), hex(bytes(xlmr_tokens[1][0], encoding='utf-16')[2]) # LEならこっち

('0x25', '0x81')

## 4.4 固有表現認識用のTransformer

- 系列全体を分類するようなテキスト分類では以下のようになっていた。
  - `[CLS]`トークンの部分に該当する隠れ層を全結合層に通すことで分類器を構成。

![](https://github.com/nokomoro3/book-ml-transformers/blob/main/img/ml-transformers-chap04-multilingal-ner_2022-08-31-07-46-57.png?raw=1)

- 固有表現認識はこれと違い、トークンごとに分類する
- 具体的には、各トークンに該当する隠れ層を、それぞれ同じ全結合層に通すことで、固有表現の結果を出力（分類）を得る。

![](https://github.com/nokomoro3/book-ml-transformers/blob/main/img/ml-transformers-chap04-multilingal-ner_2022-08-31-07-49-31.png?raw=1)

- そのため、固有表現認識はトークン分類とも呼ばれる。
- サブワードの扱い
  - BERTの論文では、サブワードには`IGN`というものを割り当てて無視している。
  - ここでもこの慣習に従う。

## 4.5 Transformer モデルクラスの詳細

- Transformersは以下のような命名規則で、タスク専用クラスを構成している。
  - `AutoModelFor<Task>`
  - `<ModelName>For<Task>`
- このアプローチには限界があり、`<Task>`が存在しないケースが実際には発生する。
- そのため本書では、`<Task>`を自身で定義する方法を示す。

### 4.5.1 ボディとヘッド

- Transformersでは、ボディだけのクラスと、ヘッドを含んだクラスで実装されている。
  - ボディだけの例
    - BertModel
    - GPT2Model
  - ヘッドを含む例
    - BertForMaskedLM
    - BertForSequenceClassification
- このような分離された構成とすることで、カスタムヘッドを自作して、モデルを構築していくことが可能。

### 4.5.2 トークン分類のためのカスタムモデルの作成

- XLM-R用のトークン分類ヘッドを構築する。
- 今回はあくまで演習のためで、実際にはトークン分類ヘッドは以下に存在する。
  - XLMRobertaForTokenClassification
- 以下がその実装である。

In [46]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

# PreTrainedModelを継承することで、from_pretrained()などのユーティリティ関数が使用可能になります。
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):

    # 標準的なXLM-Rの設定を適用
    config_class = XLMRobertaConfig

    def __init__(self, config):

        # ベースクラスであるRobertaPreTrainedModelを初期化
        # 事前学習された重みの初期化や読み込みを実施する
        super().__init__(config)

        self.num_labels = config.num_labels
        
        # モデルボディのロード
        # add_pooling_layer=Falseとすることで、[CLS]トークン以外の隠れ状態が取得できるようになる
        self.roberta = RobertaModel(config, add_pooling_layer=False)

        # トークン分類ヘッドの用意
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # 重みのロードと初期化
        # ボディに対して事前学習した重みのロードし、ヘッドをランダムに初期化する
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):

        # モデルボディを使って、エンコーダの表現を取得
        # 必要なのは、input_idsとattention_maskとなる。
        outputs = self.roberta(input_ids, attention_mask=attention_mask,
            token_type_ids=token_type_ids, **kwargs)

        # 分類器をエンコーダ表現に適用
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)

        # 損失の計算
        # labelsを与えればロスが計算される
        # attention_maskを考慮して損失を計算する場合はもう少し工夫が必要
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # モデルの出力オブジェクトを返す
        return TokenClassifierOutput(loss=loss, logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions)

### 4.5.3 カスタムモデルのロード

- データセットから取得済みのtagsを使ってラベルのマッピングをする。

In [47]:
tags

ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], names_file=None, id=None)

In [48]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

- この情報を、AutoConfigのfrom_pretrainedに引数として与える。

In [49]:
from transformers import AutoConfig

xlmr_config = AutoConfig.from_pretrained(
    xlmr_model_name,
    num_labels=tags.num_classes, id2label=index2tag, label2id=tag2index
)

- このconfigを使用して、モデルをロードする。
- 今までのようなconfigを指定しない場合は勝手に関連する設定ファイルがロードされていた。

In [50]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xlmr_model = XLMRobertaForTokenClassification\
    .from_pretrained(xlmr_model_name, config=xlmr_config)\
    .to(device)

- まずは既に読み込んでいたトークナイザーのテスト

In [51]:
input_ids = xlmr_tokenizer.encode(text, return_tensors="pt")
pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Input IDs,0,21763,37456,15555,5161,7,2356,5753,38,2


- モデルの入出力テスト
  - 固有表現のタグ数７と一致した形となっている

In [52]:
outputs = xlmr_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs, dim=-1)
print(f"Number of tokens in sequence: {len(xlmr_tokens)}")
print(f"Shape of outputs: {outputs.shape}")

Number of tokens in sequence: 10
Shape of outputs: torch.Size([1, 10, 7])


- 予測値をタグに直してみる。
  - まだランダム初期化しただけであるため、精度は良くない。

In [53]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens, preds], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Tags,O,I-PER,B-ORG,B-LOC,I-PER,I-PER,I-PER,I-PER,I-PER,O


- 一連の処理を関数に定義しておきます。

In [54]:
def tag_text(text, tags, model, tokenizer):
    # 特殊な文字列を含むトークンを取得
    tokens = tokenizer(text).tokens()
    # 系列をIDにエンコード
    input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)
    # 7つのクラス分布にわたる予測を得る
    outputs = model(input_ids)[0]
    # argmaxを使い、トークンごとにもっとも可能性の高いクラスを取得
    predictions = torch.argmax(outputs, dim=2)
    # DataFrameへ変換
    preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])